In [1]:
verbose=False

AND = lambda x1, x2: x1 and x2
OR = lambda x1, x2: x1 or x2
XOR = lambda x1, x2: 1 if x1 != x2 else 0
pad_level = lambda l: "0"*(2-len(str(l))) + str(l)

class Input:
    def __init__(self, v, name=""):
        self.v = v
        self.name=name
        self.children = []
        
    def calc(self, to_kids = False):
        if to_kids:
            for child in self.children:
                child.set_value(self.v)        
        return self.v

class Gate:
    def __init__(self, p1, p2, op, name=""):
        # a class with both parents *linked* and
        # an operation
        self.name = name
        self.p1 = p1
        self.p2 = p2
        self.op_name = op
        if op == 'AND':
            self.op = AND
        elif op == 'OR':
            self.op = OR
        elif op == 'XOR':
            self.op = XOR
        
        # current input values
        self.v1 = None
        self.v2 = None
        
        # we want to know the children to pass down also
        self.children = []
        
        # make sure the other gates know about their kids
        self.p1.children.append(self)
        self.p2.children.append(self)   
        
    def calc(self, to_kids = False):
        if self.v1 is not None and self.v2 is not None:
            out = self.op(self.v1, self.v2)
        else:
            raise ValueError("need 2 values")
        if to_kids:
            for child in self.children:
                child.set_value(out)
        return out
            
    def set_value(self, v):
        if self.v1 is None:
            self.v1 = v
        elif self.v2 is None:
            self.v2 = v
        else:
            raise ValueError("both values set")
            
    def add_child(self, gate):
        self.children.append(gate)
        
    def is_ready(self):
        return True if self.v1 is not None and self.v2 is not None else False
    
    def __repr__(self):
        return f"Gate {self.name}: {self.p1.name} {self.op_name} {self.p2.name}"
            

In [2]:
gate_confs = {}
inputs = {}
with open('input.txt', 'r') as fl:
    for ln in fl:
        if ':' in ln:
            name, val = ln.split(':')
            inputs[name] = Input(int(val.strip()), name=name)
        elif '->' in ln:
            p1, op, p2, _a, name = ln.strip().split(' ')
            gate_confs[name] = {'p1':p1, 'p2':p2, 'op':op}

In [3]:
def init_gates(gate_confs, inputs, swaps = {}):
    gates = {}
    to_init = set(gate_confs.keys())
    while len(to_init) > 0:
        for name in to_init:
            confs = gate_confs[name]
            if name in swaps:
                name = swaps[name]           
            p1, p2, op = confs['p1'], confs['p2'], confs['op']
            if (p1 in gates or p1 in inputs) and (p2 in gates or p2 in inputs):
                p1 = gates[p1] if p1 in gates else inputs[p1]
                p2 = gates[p2] if p2 in gates else inputs[p2]            
                gates[name] = Gate(p1, p2, op, name)

        to_init = set(gate_confs.keys()) - set(gates.keys()) - set(inputs.keys())
        to_init = set(list(map(lambda gt: swaps.get(gt,gt), to_init)))
    return gates
# got this via slewthing below some level at a time. 
gate_swaps = {'z21':'shh', 'shh':'z21',
              'vgs':'dtk', 'dtk':'vgs',
              'z33':'dqr', 'dqr':'z33',
              'z39':'pfw', 'pfw':'z39'}
gates = init_gates(gate_confs, inputs, gate_swaps)

In [4]:
for name, inp in inputs.items():
    out = inp.calc(to_kids=True)
    if verbose:
        print(name, out)

to_eval = set(gates.keys())

while len(to_eval) > 0:
    ran = set()
    for name in to_eval:
        gate = gates[name]
        if not gate.is_ready():
            if verbose:
                print(name, 'not ready.', gate.v1, gate.v2)
        else:
            out = gate.calc(to_kids = True)
            ran.add(name)
            if verbose:
                print(name, 'passing', out, 'to children')
    to_eval = to_eval - ran
    

In [5]:
result = {}
addx = {}
addy = {}
for name, gate in gates.items():
    if name[0]=='z':
        i = int(name[1:])
        result[i] = gate.calc()
    
for name, inp in inputs.items():
    i = int(name[1:])    
    if name[0] == 'x':
        addx[i] = inp.v
    elif name[0] == 'y':
        addy[i] = inp.v
n = max(result) + 1

In [6]:
z = int(''.join(reversed([str(result[i]) for i in range(len(result))])), base=2)
x = int(''.join(reversed([str(addx[i]) for i in range(len(addx))])), base=2)
y = int(''.join(reversed([str(addy[i]) for i in range(len(addy))])), base=2)

In [7]:
z, x, y

(42325053295186, 22044444944409, 20280608350777)

In [8]:
','.join(sorted(set(list(gate_swaps.values()) + list(gate_swaps.keys()))))

'dqr,dtk,pfw,shh,vgs,z21,z33,z39'

In [9]:
# slewthing

In [10]:
gate_swaps = {'z21':'shh', 'shh':'z21',
              'vgs':'dtk', 'dtk':'vgs',
              'z33':'dqr', 'dqr':'z33',
              'z39':'pfw', 'pfw':'z39'}
gates = init_gates(gate_confs, inputs, gate_swaps)

In [11]:
bad_gates = []

# find the first half adders
algo_gates = {}
for nm in inputs:
    if nm[0]=='y':
        level = int(nm[1:])
        if level == 0:
            op_type = 'fulladd'
        else:
            op_type = 'halfadd'
        level_inputs = ("y"+pad_level(level), "x"+pad_level(level))
        for gate in inputs[nm].children:
            if gate.p1.name in level_inputs and gate.p2.name in level_inputs:
                if gate.op_name == 'XOR':
                    if level != 0 and gate.name == 'z'+pad_level(level):
                        bad_gates.append(f'{str(level)}_{op_type}_res')
                    algo_gates[gate] = f'{str(level)}_{op_type}_res'
                elif gate.op_name == 'AND':
                    if gate.name == 'z'+pad_level(level):
                        bad_gates.append(f'{str(level)}_{op_type}_rem')                    
                    algo_gates[gate] = f'{str(level)}_{op_type}_rem'
                

In [12]:
# run this over and over until i cant then figure out how it's
# broken and figure out the next swap
inv_algo_gates = {v:k for k,v in algo_gates.items()}       
for gate, algo_step in list(algo_gates.items()):
    level, op_type, out_type = algo_step.split('_')
    if op_type == 'halfadd' and out_type == 'res':
        #print(algo_step, gate, gate.children)
        for child in gate.children:
            if child.op_name == 'XOR'and child not in algo_gates:
                prev_remainder = '_'.join((str(int(level)-1), 'fulladd', 'rem'))
                if prev_remainder in inv_algo_gates and \
                    inv_algo_gates[prev_remainder] in (child.p1, child.p2) and \
                    child.name == 'z' + pad_level(level):
                    algo_gates[child] = f'{int(level)}_fulladd_res'
                    print(child, f'{int(level)}_fulladd_res' )
                    
    if op_type == 'fulladd' and out_type == 'rem':
        #print(algo_step, gate, gate.children)
        for child in gate.children:
            next_res = f'{int(level)+1}_halfadd_res'
            if child.op_name == 'AND' and child not in algo_gates:
                if ((child.p1 in algo_gates and algo_gates[child.p1] == next_res) or \
                   (child.p2 in algo_gates and algo_gates[child.p2] == next_res)) and \
                   child.name[0] != 'z':
                    algo_gates[child] = f'{int(level)+1}_halfadd2_rem'
                    print(child, f'{int(level)+1}_halfadd2_rem')
                    
    if op_type == 'halfadd2' and out_type == 'rem':
        #print(algo_step, gate, gate.children)
        for child in gate.children:
            half_rem = f'{int(level)}_halfadd_rem'
            if child.op_name == 'OR'and child not in algo_gates:
                if ((child.p1 in algo_gates and algo_gates[child.p1] == half_rem) or \
                   (child.p2 in algo_gates and algo_gates[child.p2] == half_rem)) and \
                   child.name[0] != 'z':
                    algo_gates[child] = f'{int(level)}_fulladd_rem' 
                    print(child, f'{int(level)}_fulladd_rem' )

Gate ndb: mcg AND vcn 1_halfadd2_rem
Gate ndb: mcg AND vcn 1_halfadd2_rem
Gate z01: mcg XOR vcn 1_fulladd_res
